# Integração e preparação de dados

## Objetivo

#### O objetivo aqui é importar todas as 18 bases de dados contidas em arquivos .xlsx, integrá-las e limpá-las para utilização posterior. Para então, conectar com duas APIs para buscar informações de continente, subregião, código, Índice de Desenvolvimento Humano em 2010 e em 2020.

## 1. Importando bibliotecas e definindo funções a serem utilizadas

In [1]:
import pandas as pd
import requests    
import os

#### Verificando o arquivo em Excel, é possível perceber que a coluna que indica o ano da coleta, em alguns arquivos, tem o tipo de data(ex: 01-01-2020), porém, só precisamos do ano.

In [2]:

# Tendo em vista essa inconsistência, criaremos uma função
# para trazer somente os 4 primeiros dígitos (o ano)

def extrair_data(i):
    
    valor = i[:4]
    return valor

# Logo em seguida, outra função para importar e tratar as várias bases separadamente

def importar_arquivos(path:str):

    df = pd.read_excel(path)    # Leitura do arquivo
    df['Year'] = df['Year'].astype('str')   # Coluna ano transformada em string para extrair o ano
    df['Year'] = df['Year'].apply(extrair_data) # E agora aplicando a função criada anteriormente
    df['Year'] = df['Year'].astype('int')
    # A variável 'Year' será mantida como número inteiro 
    # para facilitar uma filtragem posterior

    return df

# Abaixo serão criadas todas as funções para conexão com as APIs

def buscar_codigo_pais(country):
    try:
        # Primeiro, tentaremos buscar por correspondência exata do nome do país
        data = requests.get(f"https://restcountries.com/v3.1/name/{country}?fullText=true").json()
        # procurando pela sigla dentro da chave abaixo
        return data[0]['cca2']
    except:
        # Caso não funcione, tentaremos uma correspondência aproximada
        try:
            data = requests.get(f"https://restcountries.com/v3.1/name/{country}").json()
            return data[0]['cca2']
        except Exception as e:
            print(f"Erro ao buscar o código para o país {country}")

# As duas funções abaixo seguem a mesma lógica, porém buscando região e subregião

def buscar_regiao(country):
    try:
        data = requests.get(f"https://restcountries.com/v3.1/name/{country}?fullText=true").json()
        return data[0]['region']
    except:
        try:
            data = requests.get(f"https://restcountries.com/v3.1/name/{country}").json()
            return data[0]['region']
        except Exception as e:
            print(f"Erro ao buscar a região para o país {country}")
            return 'Unknown'

def buscar_subregiao(country):
    try:
        data = requests.get(f"https://restcountries.com/v3.1/name/{country}?fullText=true").json()
        return data[0]['subregion']
    except:
        try:
            data = requests.get(f"https://restcountries.com/v3.1/name/{country}").json()
            return data[0]['subregion'] 
        except Exception as e:
            print(f"Erro ao buscar a subregião para o país {country}")
            return 'Unknown'

# Com a sigla dos países, buscaremos pelas informações de IDH

def buscar_idh_2010(codigo_pais):    
    try:
        api = requests.get(f'https://servicodados.ibge.gov.br/api/v1/paises/{codigo_pais}/indicadores/77831').json()
        dados_1 = api[0]['series']          # A estrutura que essa API retorna contém muitos dicionários aninhados,
        dados_2 = dados_1[0]['serie']       # então tive que atribuir os valores sequencialmente em variáveis
        dados_final = dados_2[-24]['2010']  # até encontrar o valor do ano que estamos buscando
        return dados_final
    except Exception as e:
        print(f"Erro ao buscar o idh de 2010 para o país {codigo_pais}")
        return 'Unknown'

def buscar_idh_2020(codigo_pais):    
    try:
        api = requests.get(f'https://servicodados.ibge.gov.br/api/v1/paises/{codigo_pais}/indicadores/77831').json()
        dados_1 = api[0]['series']
        dados_2 = dados_1[0]['serie']
        dados_final = dados_2[-3]['2020']
        return dados_final
    except Exception as e:
        print(f"Erro ao buscar o idh de 2020 para o país {codigo_pais}")
        return 'Unknown'


## 2. Integração da base

#### Todas as planilhas serão consolidadas em um único dataframe, para que o tratamento seja realizado uma única vez

In [3]:
caminho = 'Dados/'  # Diretório
arquivos = os.listdir(caminho) # Nomes de todos os arquivos da pasta

dfs = []

for arquivo in arquivos:                                    # Para cada arquivo na pasta, 
    if arquivo[-4:] == "xlsx":                              # é verificado se a extensão é .xlsx
        dfs.append(importar_arquivos(caminho+arquivo))      # se sim, a função criada acima é aplicada

df_final = pd.concat(dfs, axis = 0)  # E as linhas desses dataframes são integradas / empilhadas

print(df_final['Year'].unique()) # Verificação se todos os anos foram transformados

df_final

[2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019
 2020 2021 2022 2023]


,Country,Year,Rank,Total,C1: Security Apparatus,C2: Factionalized Elites,C3: Group Grievance,E1: Economy,E2: Economic Inequality,E3: Human Flight and Brain Drain,P1: State Legitimacy,P2: Public Services,P3: Human Rights,S1: Demographic Pressures,S2: Refugees and IDPs,X1: External Intervention,Change from Previous Year
0,Sudan,2006,1st,112.3,9.8,9.1,9.7,7.5,9.2,9.1,9.5,9.5,9.8,9.6,9.7,9.8,NaN
1,Congo Democratic Republic,2006,2nd,110.1,9.8,9.6,9.1,8.1,9.0,8.0,9.0,9.0,9.5,9.5,9.5,10.0,NaN
2,Cote d'Ivoire,2006,3rd,109.2,9.8,9.8,9.8,9.0,8.0,8.5,10.0,8.5,9.4,8.8,7.6,10.0,NaN
3,Iraq,2006,4th,109.0,9.8,9.7,9.8,8.2,8.7,9.1,8.5,8.3,9.7,8.9,8.3,10.0,NaN
4,Zimbabwe,2006,5th,108.9,9.4,8.5,8.5,9.8,9.2,9.0,8.9,9.5,9.5,9.7,8.9,8.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,Switzerland,2023,175th,17.8,1.4,1.0,2.1,1.6,2.4,1.0,0.3,1.6,0.4,2.4,3.2,0.4,NaN
175,New Zealand,2023,176th,16.7,1.6,1.4,2.0,2.6,2.6,1.6,0.5,1.1,0.5,1.1,1.2,0.5,NaN
176,Finland,2023,177th,16.0,2.0,1.4,0.3,2.7,1.6,1.5,0.4,1.0,0.5,1.7,1.9,1.0,NaN
177,Iceland,2023,178th,15.7,0.4,1.8,0.5,2.6,1.5,1.6,0.4,0.9,0.4,1.5,1.5,2.6,NaN


## 3. Limpeza

#### Verificando valores nulos

In [4]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3170 entries, 0 to 178
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country                           3170 non-null   object 
 1   Year                              3170 non-null   int32  
 2   Rank                              3170 non-null   object 
 3   Total                             3170 non-null   float64
 4   C1: Security Apparatus            3170 non-null   float64
 5   C2: Factionalized Elites          3170 non-null   float64
 6   C3: Group Grievance               3170 non-null   float64
 7   E1: Economy                       3170 non-null   float64
 8   E2: Economic Inequality           3170 non-null   float64
 9   E3: Human Flight and Brain Drain  3170 non-null   float64
 10  P1: State Legitimacy              3170 non-null   float64
 11  P2: Public Services               3170 non-null   float64
 12  P3: Human Ri

#### A variável que contém as variações anuais, contém muitos valores vazios, e como é uma informação que podemos conseguir facilmente, é mais fácil removê-la.

In [5]:
df_final.drop(columns = 'Change from Previous Year', inplace = True)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3170 entries, 0 to 178
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country                           3170 non-null   object 
 1   Year                              3170 non-null   int32  
 2   Rank                              3170 non-null   object 
 3   Total                             3170 non-null   float64
 4   C1: Security Apparatus            3170 non-null   float64
 5   C2: Factionalized Elites          3170 non-null   float64
 6   C3: Group Grievance               3170 non-null   float64
 7   E1: Economy                       3170 non-null   float64
 8   E2: Economic Inequality           3170 non-null   float64
 9   E3: Human Flight and Brain Drain  3170 non-null   float64
 10  P1: State Legitimacy              3170 non-null   float64
 11  P2: Public Services               3170 non-null   float64
 12  P3: Human Ri

#### Além do sufixos, também existe um registro do ano de 2012 que não entrou no ranking, então ele será excluído para não prejudicar posteriormente

In [6]:
print(df_final['Rank'].unique())
df_final[df_final['Rank'] == 'n/r'].head()

['1st' '2nd' '3rd' '4th' '5th' '6th' '8th' '9th' '10th' '11th' '13th'
 '14th' '15th' '16th' '18th' '19th' '20th' '21st' '22nd' '24th' '25th'
 '26th' '27th' '28th' '29th' '30th' '31st' '32nd' '33rd' '35th' '36th'
 '37th' '39th' '41st' '42nd' '43rd' '44th' '45th' '46th' '47th' '49th'
 '50th' '51st' '52nd' '54th' '55th' '56th' '57th' '59th' '60th' '61st'
 '63rd' '65th' '66th' '67th' '68th' '70th' '71st' '72nd' '73rd' '74th'
 '75th' '76th' '77th' '78th' '79th' '80th' '81st' '82nd' '83rd' '84th'
 '85th' '86th' '87th' '88th' '89th' '90th' '91st' '92nd' '93rd' '94th'
 '95th' '96th' '97th' '98th' '100th' '101st' '102nd' '103rd' '104th'
 '105th' '106th' '107th' '108th' '109th' '110th' '111th' '112th' '113th'
 '114th' '115th' '116th' '117th' '118th' '119th' '120th' '121st' '122nd'
 '123rd' '124th' '125th' '126th' '127th' '128th' '129th' '130th' '131st'
 '132nd' '133rd' '134th' '135th' '136th' '137th' '138th' '139th' '140th'
 '141st' '142nd' '143rd' '144th' '146th' '7th' '12th' '17th' '23rd' '38t

,Country,Year,Rank,Total,C1: Security Apparatus,C2: Factionalized Elites,C3: Group Grievance,E1: Economy,E2: Economic Inequality,E3: Human Flight and Brain Drain,P1: State Legitimacy,P2: Public Services,P3: Human Rights,S1: Demographic Pressures,S2: Refugees and IDPs,X1: External Intervention
3,South Sudan,2012,n/r,108.4,9.7,10.0,10.0,7.3,8.8,6.4,9.1,9.5,9.2,8.4,9.9,10.0


In [7]:
linhas_antes = df_final.shape[0]
df_final = df_final.drop(df_final[df_final['Rank'] == 'n/r'].index)
linhas_depois = df_final.shape[0]
linhas_removidas = linhas_antes - linhas_depois
print('Linhas antes: ',linhas_antes)
print('Linhas removidas: ', linhas_removidas)
print('Linhas depois: ', linhas_depois)


Linhas antes:  3170
Linhas removidas:  18
Linhas depois:  3152


#### Nessa mesma variável, existe também um sufixo que impossibilita a transformação em números

In [8]:
df_final['Rank'] = df_final['Rank'].str[:-2] # Então removeremos esse sufixo
df_final['Rank'] = df_final['Rank'].astype('int') # E converteremos a variável para número
df_final['Rank'].unique()

array([  1,   2,   3,   5,   6,   8,   9,  10,  11,  13,  14,  15,  16,
        18,  19,  20,  21,  22,  24,  25,  26,  27,  28,  29,  30,  31,
        32,  33,  35,  36,  37,  39,  41,  42,  43,  44,  45,  46,  47,
        49,  50,  51,  52,  54,  55,  56,  57,  59,  60,  61,  63,  65,
        66,  67,  68,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 146,
         7,  12,  17,  23,  38,  40,  48,  53,  58,  62,  69, 145, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176,  34,  64, 157, 177,  99,   4, 178, 179])

#### Agora que a variável rank é numérica, poderemos usá-la para ordenar a base de dados por ano e rank

In [9]:
df_final.sort_values(by=['Year','Rank'],ascending = True, inplace = True)
df_final.head()

,Country,Year,Rank,Total,C1: Security Apparatus,C2: Factionalized Elites,C3: Group Grievance,E1: Economy,E2: Economic Inequality,E3: Human Flight and Brain Drain,P1: State Legitimacy,P2: Public Services,P3: Human Rights,S1: Demographic Pressures,S2: Refugees and IDPs,X1: External Intervention
0,Sudan,2006,1,112.3,9.8,9.1,9.7,7.5,9.2,9.1,9.5,9.5,9.8,9.6,9.7,9.8
1,Congo Democratic Republic,2006,2,110.1,9.8,9.6,9.1,8.1,9.0,8.0,9.0,9.0,9.5,9.5,9.5,10.0
2,Cote d'Ivoire,2006,3,109.2,9.8,9.8,9.8,9.0,8.0,8.5,10.0,8.5,9.4,8.8,7.6,10.0
4,Zimbabwe,2006,5,108.9,9.4,8.5,8.5,9.8,9.2,9.0,8.9,9.5,9.5,9.7,8.9,8.0
5,Chad,2006,6,105.9,9.4,9.5,8.5,7.9,9.0,8.0,9.5,9.0,9.1,9.0,9.0,8.0


#### Os nomes dos indicadores possuem prefixos que dificultam a digitação de seus nomes no código, então optei por removê-los

In [10]:
colunas = df_final.columns.to_list() # Criação de uma lista com os nomes de colunas do dataframe final

# Para facilitar nas análises, removerei os prefixos das colunas de cada indicador
colunas_new = []

for coluna in colunas:                  # Esse loop fará a verificação de todos os nomes
    if coluna[2] != ":":                # se o terceiro caractere do nome não for ":"
        colunas_new.append(coluna)      # O nome completo será adicionado à nova lista 
    else:
        colunas_new.append(coluna[4:])  # Caso contrário, o nome será utilizado a partir do 5º caractere
       
df_final.columns = colunas_new
df_final.head()

,Country,Year,Rank,Total,Security Apparatus,Factionalized Elites,Group Grievance,Economy,Economic Inequality,Human Flight and Brain Drain,State Legitimacy,Public Services,Human Rights,Demographic Pressures,Refugees and IDPs,External Intervention
0,Sudan,2006,1,112.3,9.8,9.1,9.7,7.5,9.2,9.1,9.5,9.5,9.8,9.6,9.7,9.8
1,Congo Democratic Republic,2006,2,110.1,9.8,9.6,9.1,8.1,9.0,8.0,9.0,9.0,9.5,9.5,9.5,10.0
2,Cote d'Ivoire,2006,3,109.2,9.8,9.8,9.8,9.0,8.0,8.5,10.0,8.5,9.4,8.8,7.6,10.0
4,Zimbabwe,2006,5,108.9,9.4,8.5,8.5,9.8,9.2,9.0,8.9,9.5,9.5,9.7,8.9,8.0
5,Chad,2006,6,105.9,9.4,9.5,8.5,7.9,9.0,8.0,9.5,9.0,9.1,9.0,9.0,8.0


### A fonte dos dados indica que são 179 países, então irei verificar se a quantidade de países diferentes condizem com essa informação

In [11]:
len(df_final['Country'].unique())

231

#### Retirando os excessos de espaços antes ou após o texto

In [12]:
df_final['Country'] = df_final['Country'].str.strip()
len(df_final['Country'].unique())

187

#### Apenas com o tratamento anterior, conseguimos padronizar os nomes de 44 países, porém ainda faltam 8, precisaremos verificar manualmente se existem países com nomes parecidos

In [13]:
df_final['Country'].unique() 

array(['Sudan', 'Congo Democratic Republic', "Cote d'Ivoire", 'Zimbabwe',
       'Chad', 'Somalia', 'Haiti', 'Pakistan', 'Afghanistan', 'Guinea',
       'Liberia', 'Central African Republic', 'North Korea', 'Burundi',
       'Sierra Leone', 'Yemen', 'Myanmar', 'Bangladesh', 'Nepal',
       'Uganda', 'Nigeria', 'Uzbekistan', 'Rwanda', 'Sri Lanka',
       'Ethiopia', 'Colombia', 'Kyrgyz Republic', 'Malawi',
       'Burkina Faso', 'Egypt', 'Indonesia', 'Kenya', 'Syria',
       'Bosnia and Herzegovina', 'Cameroon', 'Angola', 'Togo', 'Bhutan',
       'Laos', 'Mauritania', 'Tajikistan', 'Russia', 'Niger',
       'Turkmenistan', 'Guinea Bissau', 'Cambodia', 'Dominican Republic',
       'Papua New Guinea', 'Belarus', 'Guatemala', 'Equatorial Guinea',
       'Iran', 'Eritrea', 'Serbia', 'Bolivia', 'China', 'Moldova',
       'Nicaragua', 'Georgia', 'Azerbaijan', 'Cuba', 'Ecuador',
       'Venezuela', 'Lebanon', 'Zambia', 'Israel and West Bank', 'Peru',
       'Philippines', 'Vietnam', 'Tanzania'

In [14]:
df_final['Country'] = df_final['Country'].replace('Congo Republic','Congo')
df_final['Country'] = df_final['Country'].replace('Congo Democratic Republic','Congo')
df_final['Country'] = df_final['Country'].replace("Côte d'Ivoire","Ivory Coast")
df_final['Country'] = df_final['Country'].replace("Cote d'Ivoire","Ivory Coast")
df_final['Country'] = df_final['Country'].replace('Czechia','Czech Republic')
df_final['Country'] = df_final['Country'].replace('Cape Verde','Cabo Verde')
df_final['Country'] = df_final['Country'].replace('Equatorial Guinea','Guinea')
df_final['Country'] = df_final['Country'].replace('Guinea Bissau','Guinea-Bissau')
df_final['Country'] = df_final['Country'].replace('Israel and West Bank','Israel')
df_final['Country'] = df_final['Country'].replace('Kyrgyz Republic', 'Kyrgyzstan')
df_final['Country'] = df_final['Country'].replace('Slovak Republic','Slovakia')
len(df_final['Country'].unique())

179

## 4. Conexão com a API

#### Agora que já temos os 179 países que são descritos oficialmente, criaremos um dataframe com os nomes de países distintos para buscar o continente através de uma API

In [15]:
paises = pd.DataFrame(df_final['Country'].unique()) # Trazendo os nomes de países distintos para um dataframe
paises.columns = ['Country']
paises = paises.sort_values(by = 'Country', ascending = True) # Ordenando alfabeticamente,
paises.index = range(0,len(paises))                           # consertando o índice
paises['Country'].str.strip()                                 # e removendo possíveis excessos de espaçamento
paises

,Country
0,Afghanistan
1,Albania
2,Algeria
3,Angola
4,Antigua and Barbuda
...,...
174,Venezuela
175,Vietnam
176,Yemen
177,Zambia


#### Então o próximo passo seria aplicar as funções que foram criadas no ínicio

In [16]:
# Usaremos o nome do país para buscar sigla, primeiramente 
# (Aparentemente, ao aplicar as 3 funções em conjunto, o código parece rodar em loop)
paises['ID'] = paises['Country'].apply(buscar_codigo_pais)


In [17]:
paises['Region'] = paises['Country'].apply(buscar_regiao)
paises['Subregion'] = paises['Country'].apply(buscar_subregiao)

In [18]:
paises['ID'] = paises['ID'].str.strip()
# Apenas por precaução, já que a variável será utilizada para pesquisa

In [19]:
# E com a sigla que conseguimos, pesquisaremos em outra API o restante do que precisamos
paises['IDH_2010'] = paises['ID'].apply(buscar_idh_2010)
paises['IDH_2020'] = paises['ID'].apply(buscar_idh_2020)

Erro ao buscar o idh de 2010 para o país KP
Erro ao buscar o idh de 2010 para o país PS
Erro ao buscar o idh de 2010 para o país SO
Erro ao buscar o idh de 2020 para o país KP
Erro ao buscar o idh de 2020 para o país PS
Erro ao buscar o idh de 2020 para o país SO


In [20]:
paises[paises['ID'].isin(['KP','PS','SO'])]

,Country,ID,Region,Subregion,IDH_2010,IDH_2020
118,North Korea,KP,Asia,Eastern Asia,Unknown,Unknown
123,Palestine,PS,Asia,Western Asia,Unknown,Unknown
146,Somalia,SO,Africa,Eastern Africa,Unknown,Unknown


#### Após pesquisas na internet, e na própria API, é possível notar que os países listados não possuem métricas de IDH, então os removeremos do dataframe principal; e também removeremos os anos que não usaremos na análise

In [21]:
# Trazendo todos os dados que foram recuperados para o dataframe principal
df_final = df_final.merge(paises, how = 'left', on = 'Country') 
df_final.head(10)

,Country,Year,Rank,Total,Security Apparatus,Factionalized Elites,Group Grievance,Economy,Economic Inequality,Human Flight and Brain Drain,...,Public Services,Human Rights,Demographic Pressures,Refugees and IDPs,External Intervention,ID,Region,Subregion,IDH_2010,IDH_2020
0,Sudan,2006,1,112.3,9.8,9.1,9.7,7.5,9.2,9.1,...,9.5,9.8,9.6,9.7,9.8,SD,Africa,Northern Africa,0.486,0.51
1,Congo,2006,2,110.1,9.8,9.6,9.1,8.1,9.0,8.0,...,9.0,9.5,9.5,9.5,10.0,CD,Africa,Middle Africa,0.429,0.479
2,Ivory Coast,2006,3,109.2,9.8,9.8,9.8,9.0,8.0,8.5,...,8.5,9.4,8.8,7.6,10.0,CI,Africa,Western Africa,0.473,0.551
3,Zimbabwe,2006,5,108.9,9.4,8.5,8.5,9.8,9.2,9.0,...,9.5,9.5,9.7,8.9,8.0,ZW,Africa,Southern Africa,0.512,0.6
4,Chad,2006,6,105.9,9.4,9.5,8.5,7.9,9.0,8.0,...,9.0,9.1,9.0,9.0,8.0,TD,Africa,Middle Africa,0.362,0.397
5,Somalia,2006,6,105.9,10.0,9.8,8.0,8.5,7.5,7.0,...,10.0,9.5,9.0,8.1,8.5,SO,Africa,Eastern Africa,Unknown,Unknown
6,Haiti,2006,8,104.6,9.4,9.6,8.8,8.4,8.3,8.0,...,9.3,9.6,8.8,5.0,10.0,HT,Americas,Caribbean,0.433,0.54
7,Pakistan,2006,9,103.1,9.1,9.1,8.6,7.0,8.9,8.1,...,7.5,8.5,9.3,9.3,9.2,PK,Asia,Southern Asia,0.505,0.543
8,Afghanistan,2006,10,99.8,8.2,8.0,9.1,7.5,8.0,7.0,...,8.0,8.2,7.9,9.6,10.0,AF,Asia,Southern Asia,0.448,0.483
9,Guinea,2006,11,99.0,8.1,9.0,8.1,8.0,8.0,8.4,...,9.0,8.1,7.5,7.2,8.5,GN,Africa,Western Africa,0.415,0.466


#### Aqui é aplicado o filtro de países

In [22]:
print('Países a serem excluídos: ',df_final[df_final['IDH_2010'] == 'Unknown']['Country'].unique())
print('Linhas do dataframe antes: ', df_final.shape[0])
print('Linhas a serem retiradas: ',df_final[df_final['IDH_2010'] == 'Unknown']['Country'].shape[0])
df_final = df_final[df_final['IDH_2010'] != 'Unknown']
print('Linhas do dataframe depois: ', df_final.shape[0])

Países a serem excluídos:  ['Somalia' 'North Korea' 'Palestine']
Linhas do dataframe antes:  3152
Linhas a serem retiradas:  39
Linhas do dataframe depois:  3113


#### E aqui, o filtro de anos

In [23]:
print('Linhas antes do filtro: ', df_final.shape[0])
df_final['Year'] = df_final['Year'].astype('int')

# Usaremos o range ao invés de digitar manualmente os critérios
df_final = df_final[df_final['Year'].isin(range(2010,2021))] 

print('Anos mantidos: ', df_final['Year'].unique())
print('Linhas mantidas: ',df_final[df_final['Year'].isin(range(2010,2021))].shape[0])

Linhas antes do filtro:  3113
Anos mantidos:  [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020]
Linhas mantidas:  1923


#### Alteraremos a ordem das colunas, para manter as variáveis categóricas no início

In [25]:
df_final = df_final.reindex(columns = ['Region', 'Subregion','ID','Country', 'Year', 
                                       'Rank', 'Total', 'Security Apparatus',
                                       'Factionalized Elites', 'Group Grievance', 'Economy',
                                       'Economic Inequality', 'Human Flight and Brain Drain',
                                       'State Legitimacy', 'Public Services', 'Human Rights',
                                       'Demographic Pressures', 'Refugees and IDPs', 
                                       'External Intervention'])
df_final

,Region,Subregion,ID,Country,Year,Rank,Total,Security Apparatus,Factionalized Elites,Group Grievance,Economy,Economic Inequality,Human Flight and Brain Drain,State Legitimacy,Public Services,Human Rights,Demographic Pressures,Refugees and IDPs,External Intervention
674,Africa,Middle Africa,TD,Chad,2010,2,113.300000,9.9,9.8,9.8,8.50000,9.3,8.3,9.9,9.600000,9.6,9.400000,9.5,9.7
675,Africa,Northern Africa,SD,Sudan,2010,3,111.800000,9.8,9.9,9.9,6.70000,9.5,8.7,9.9,9.300000,9.9,8.800000,9.8,9.6
676,Africa,Middle Africa,CD,Congo,2010,5,109.900000,9.8,8.9,8.6,8.70000,9.5,8.0,8.8,9.000000,9.4,9.900000,9.6,9.7
677,Asia,Southern Asia,AF,Afghanistan,2010,6,109.300000,9.7,9.4,9.7,8.30000,8.2,7.2,10.0,8.900000,9.2,9.500000,9.2,10.0
678,Asia,Western Asia,IQ,Iraq,2010,7,107.300000,9.5,9.6,9.3,7.60000,8.8,9.3,9.0,8.400000,9.1,8.500000,8.7,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2613,Europe,Northern Europe,IS,Iceland,2020,174,17.800000,1.0,1.8,0.7,2.80000,1.0,2.2,0.7,0.700000,0.7,1.000000,1.7,3.5
2614,Europe,Northern Europe,DK,Denmark,2020,175,17.213587,1.6,1.4,4.0,1.30000,0.9,1.6,0.6,0.913587,1.4,1.300000,1.7,0.5
2615,Europe,Western Europe,CH,Switzerland,2020,176,17.094086,1.4,1.0,3.0,1.60000,1.5,1.4,0.5,1.052499,1.1,1.241587,2.8,0.5
2616,Europe,Northern Europe,NO,Norway,2020,177,16.191210,1.8,1.1,3.6,1.69121,0.7,1.0,0.5,1.100000,0.6,0.900000,2.5,0.7


#### E por fim, corrigiremos o índice

In [26]:
df_final.index = range(0,df_final.shape[0])  # Ele irá de zero até a quantidade total de linhas
df_final.head()

,Region,Subregion,ID,Country,Year,Rank,Total,Security Apparatus,Factionalized Elites,Group Grievance,Economy,Economic Inequality,Human Flight and Brain Drain,State Legitimacy,Public Services,Human Rights,Demographic Pressures,Refugees and IDPs,External Intervention
0,Africa,Middle Africa,TD,Chad,2010,2,113.3,9.9,9.8,9.8,8.5,9.3,8.3,9.9,9.6,9.6,9.4,9.5,9.7
1,Africa,Northern Africa,SD,Sudan,2010,3,111.8,9.8,9.9,9.9,6.7,9.5,8.7,9.9,9.3,9.9,8.8,9.8,9.6
2,Africa,Middle Africa,CD,Congo,2010,5,109.9,9.8,8.9,8.6,8.7,9.5,8.0,8.8,9.0,9.4,9.9,9.6,9.7
3,Asia,Southern Asia,AF,Afghanistan,2010,6,109.3,9.7,9.4,9.7,8.3,8.2,7.2,10.0,8.9,9.2,9.5,9.2,10.0
4,Asia,Western Asia,IQ,Iraq,2010,7,107.3,9.5,9.6,9.3,7.6,8.8,9.3,9.0,8.4,9.1,8.5,8.7,9.5


## 5. Exportando a base integrada para .csv

In [27]:
df_final.to_csv('FSI-INTEGRADO.csv', sep = ";",index = False)
paises.to_csv('Paises.csv', sep = ";",index = False)